# 2D Jet SPOD

In this tutorial we will explore a small dataset provided with this package that contains pressure data of the flow exiting a nozzle (also referred to as a jet).
Cylindrical coordinates (r,x) are used and they are equally spaced. 
In particular, starting from a database of pre computed solutions, we want to:
- extract the SPOD (coherent in space and time) modes
- compute the coefficients projecting the data on the SPOD basis built by gathering the modes
- learn the latent dynamics through a LSTM-based neural network
- predict future coefficients, i.e. the future evolution in time of the flow field
- compare predictions with the actual flow realizations

In detail, the starting dataset consists of 1000 flow realizations which represent the pressure field at different time instant. The time step is 12 hours.

## Loading libraries and data

The dataset is part of the data used for the regression tests that come with this library and is stored into `tests/data/fluidmechanic_data.mat`. The first step to anlyze this dataset is to import the required libraries, including the custom libraries
- 'from pyspod.spod_low_storage import SPOD_low_storage'
- 'from pyspod.spod_low_ram     import SPOD_low_ram'
- 'from pyspod.spod_streaming   import SPOD_streaming'
- 'from pyspod.emulation   import Emulation'
- 'import pyspod.utils_weights as utils_weights'
- 'import pyspod.utils as utils'.  

The first three contain different implementations of the SPOD algorithm, the first requiring low storage memory (intended for large RAM machines or small amount of data), the second requiring low RAM (intended for large dataset or small RAM machines), and the third being a streaming algorithm, that required little amount of memory (both storage and RAM) but runs typically slower than the other two.



In [ ]:
# python libraries
import os
import sys
import h5py
import shutil
import numpy as np

from pathlib import Path

# Current, parent and file paths import sys
CWD = os.getcwd()
sys.path.insert(0, os.path.join(CWD, "../../../../"))

# Import specific libraries
from pyspod.spod_low_storage import SPOD_low_storage
from pyspod.spod_low_ram     import SPOD_low_ram
from pyspod.spod_streaming   import SPOD_streaming
from pyspod.emulation   import Emulation
import pyspod.utils_weights as utils_weights
import pyspod.utils as utils  


The second step conists into importing data from "fluidmechanics_data.mat" located in the folder pysod/test/data and store them in a dictionary. 

In [ ]:
file = os.path.join(CWD,'../../../../tests/data','fluidmechanics_data.mat')
variables = ['p']
with h5py.File(file, 'r') as f:
	data_arrays = dict()
	for k, v in f.items():
		data_arrays[k] = np.array(v)

## Define global variables and global parameters

The data are stored in a matrix `X` and, to be suitable to the `PySPOD` library, it must have the following features:
- first dimension must correspond to the number of time snapshots (1000 in our case)
- last dimension should corresponds to the number of variables (1 in our case)
- the remaining dimensions corresponds to the spatial dimensions (20, and 88 in our case, that correspond to radial and axial spatial coordinates).
We note that in the present test case the data matrix `X` used is already in a shape that is suitable to `PySPOD`, as its dimension is:
$$\text{$X$ dimensions} = 1000 \times 20 \times 88 $$


It is important to note at this point that we loaded all the required data into RAM, and stored it into a `numpy.ndarray`. We will later pass this array to the constructor of the `PySPOD` class for running our analysis. However, we could have used a different approach to load the data. In fact, the constructor to the `PySPOD` class accepts an argument called `data_handler`, that points to a function whose objective is to read the data at run time. This is particularly useful for large datasets, where it might be not possible to load all the data in RAM upfront. Therefore, in this case, we could simply define a data reader function as the following:

```
def read_data(data, t_0, t_end, variables): 
    ... implement here your method
    data: path to the data file
    t_0: start time slicing
    t_end: end time slicing
    variables: list with names of the variables

    return X
```

and pass it to the `PySPOD` constructor under the argument `data_handler`. The path to the data file, will then be specified in place of the data, under the argument `X`. See below, when we setup the analysis and call the constructor for a more detailed explantion of the parameters `X` and `data_handler`. In summary, if `X` is a numpy.ndarray containing your data, `data_handler` is set to `False`, if `X` is a `str` containing the path to your data file, `data_handler` is a function that reads your data, and whose arguments must be: (1.) `str` containing the path to the data file, (2) `int` containing the start time snapshot for slicing the data sequentially at run time, (3) `int` containing the end time snapshot for slicing the data sequentially at run time, and (4) a `list` containing the name of the variables in your data file. 



The required parameters are as follows:
- `time_step`: time-sampling of the data (for now this must be constant)
- `n_snapshots`: number of time snapshots
- `block_dimension`: number of snapshots in each block, it is related to the maximum number of frequency one can extract: n_freq=n_block/{2}+1
- `n_space_dims`: number of spatial dimensions
- `n_variables`: number of variables
- `n_DFT`: length of FFT blocks

The optional parameters are as follows:
- `overlap`: dimension of the overlap region between adjacent blocks in percentage (0 to 100)
- `mean_type`: type of mean to be subtracted from the data (`longtime`, `blockwise` or `zero`)
- `normalize_weights`: weights normalization by data variance
- `normalize_data`: normalize data by variance
- `n_modes_save`: number of modes to be saved
- `conf_level`: calculate confidence level of modes
- `reuse_blocks`: whether to attempt reusing FFT blocks previously computed (if found)
- `savefft`: save FFT blocks to reuse them in the future (to save time)
- `savedir`: where to save the data

**Note that we do not set any parameter for the Weights adopted to compute th einner product in the SPOD calculation. In this case, the algorithm will use automatically uniform weighting (weighting equal 1), and it will prompt a warning stating the use of default uniform weighting.** 

In the dictionary 'params_emulation' the following variables, which allow to define some relevant parameters of a single layer neural network, are stored:
- `network`: string, type of neural network. In the present tutorial 'lstm'
- `epochs`: integer, number of epochs
- `batch_size`: integer, batch size
- `n_seq_in`: integer, dimension of input sequence 
- `n_seq_out`: integer, number of steps to predict
- `n_neurons`: number of neurons in each layer
- `dropout`: value of the dropout
- `savedir`: string, name of the directory where results will be saved

In the present test case we use 60 (`n_seq_in`=60) previous values of the coefficients in order to evaluate the next one (`n_seq_out`=1).

In [ ]:
dt = data_arrays['dt'][0,0]
block_dimension = 64 * dt
X = data_arrays[variables[0]].T
t = dt * np.arange(0,X.shape[0]); t = t.T
nt = t.shape[0]
x1 = data_arrays['r'].T; x1 = x1[:,0]
x2 = data_arrays['x'].T; x2 = x2[0,:]


trainingDataRatio = 0.8 # ratio between training data number and total number of snapshots
testingDataRatio = (1-trainingDataRatio)

# parameters
params = dict()

# -- required parameters
params['time_step'   	   ] = dt 						# data time-sampling
params['n_space_dims'    ] = 2							# number of spatial dimensions (longitude and latitude)
params['n_variables'     ] = 1							# number of variables
params['n_DFT'           ] = np.ceil(block_dimension / dt) # length of FFT blocks
# -- optional parameters
params['overlap'          ] = 50					# dimension in percentage (1 to 100) of block overlap
params['mean_type'        ] = 'blockwise' # type of mean to subtract to the data
params['normalize_weights'] = False	 			# normalization of weights by data variance
params['normalize_data'   ] = False  			# normalize data by data variance
params['n_modes_save'     ] = 8  		# modes to be saved
params['conf_level'       ] = 0.95   			# calculate confidence level
params['savedir'          ] = os.path.join(CWD, 'results', Path(file).stem)
params['reuse_blocks'] = False

params_emulation = dict()

params_emulation['network'     ] = 'lstm' 						# type of network
params_emulation['epochs'      ] = 130 						# number of epochs
params_emulation['batch_size'  ] = 32							# batch size
params_emulation['n_seq_in'    ] = 60							# dimension of input sequence 
params_emulation['n_seq_out'   ] = 1                          # number of steps to predict
params_emulation['n_neurons'   ] = 25                          # number of neurons
params_emulation['dropout'   ] = 0.15                          # dropout
params_emulation['savedir'     ] = os.path.join(CWD, 'results', Path(file).stem)


## Compute SPOD modes and coefficients 

The following lines of code are used for the initialization of variables. `X_train` and `X_test` are numpy data structures which contain the training set and the testing set respectively; therefore their dimensions are (`nt_train`, 20, 88) and (`nt_test`, 20, 88), being in this test case `nt_train`= $0.8\cdot nt$ and nt_test=$0.2\cdot nt$. 

In [ ]:
#  training and testing database definition
nt_train = int(trainingDataRatio * nt)
X_train = X[:nt_train,:,:]
nt_test = nt - nt_train
X_test  = X[nt_train:,:,:]


After we have initialized the variables above, we are ready to perform the SPOD analysisl at the end of this pahse we will have the SPOD modes computed for different ranges of frequencies.
This phase can be computationally expansive and different techniques have been developed in order to handle data.
If, as `data_handler`, we pass `False`, then we need to load the entire matrix of data into RAM, and that must comply with the **PySPOD** input data requirements (i.e. the dimension of the data matrix must correspond to (time $\times$ spatial dimension shape $\times$ number of variables). 

The arguments to the constructor are defined as follows:
  - `params`: must be a dictionary and contains the parameters that we have just defined. 
  - `data_handler`: can be either `False` or a function handler. If it is a function handler, it must hold the function to read the data. The template for the function to read the data must have as first argument the data file, as second and third the time indices through which we will slice the data in time, and as fourth argument a list containing the name of the variables. See our data reader as an example and modify it according to your needs.
  - `variables`: is a list containing our variables. 
  
 The `fit()` method returns a `PySPOD` object containg the results, the input arguments are a numpy array `X_train` and its first dimension (`nt`)

In [ ]:
# SPOD analysis
SPOD_analysis = SPOD_low_storage(
	params=params, 
	data_handler=False, 
	variables=variables
	)

# fit 
spod = SPOD_analysis.fit(X_train, nt=nt_train)

In the transform function the pressure fluctuations are computed by subtracting the mean field from the snapshots. Then the SPOD modes are ordered by frequency and the coefficients are obtained by projecting the snapshots representing the pressure fluctuations onto the reduced POD basis obtained by gathering the most significant modes. In details, the `spod.transform` function accept as input
- data: dataset on which the analysis is performed
- nt: number of snapshots of the dataset 'data'

and it returns a dictionary which contains the following keywords:
- time_mean: the average in time of snapshots
- phi_tilde: the first most significant modes ordered by frequency. These modes identify a reduced basis, significant for the case at hand.
- coeffs: the coefficients obtained by projecting the snapshots onto the SPOD basis
- reconstructed_data: snapshots reconstructed by superimposing the modes multiplied by the coefficients

The same function is used for the testing set; the testing coefficients are obtaied by projecting the pressure fluctuations snapshots of`X_test` onto the SPOD basis previously computed.

In [ ]:
# transform
coeffs_train = spod.transform(X_train, nt=nt_train, T_lb=None, T_ub=None)
coeffs_test  = spod.transform(X_test , nt=nt_test, T_lb=None, T_ub=None)

## Learning the latent space dynamics

The following lines are required in order to initialize the data structures needed to train the neural network and to store its output 

In [ ]:
# initialization of variables and structures
n_modes = params['n_modes_save'] 
n_freq = spod._n_freq_r
n_feature = coeffs_train['coeffs'].shape[0]

data_train = np.zeros([n_freq,coeffs_train['coeffs'].shape[1]],dtype='complex')
data_test = np.zeros([n_freq,coeffs_test['coeffs'].shape[1]],dtype='complex')
coeffs_emul = np.zeros([coeffs_test['coeffs'].shape[0],coeffs_test['coeffs'].shape[1]],dtype='complex')
coeffs_tmp = np.zeros([n_freq,coeffs_test['coeffs'].shape[1]],dtype='complex')


The coefficients previously evaluated can now be used for training a LSTM-based neural network.
The Emulation constructor requires the following parameters:
- params_emulation: dict containing the parameters described in the previous sections. They contain all the relevant data for creating a single-layer neural network with Dropout

The neural network is initialized by calling `spod_emulation.model_initialize` that requires the data set which the network will be trained with.

In [ ]:
# LSTM
spod_emulation = Emulation(params_emulation)

# initialization of the network
spod_emulation.model_initialize(data=data_train)


In this test case, we train `n_modes` separate neural networks, each of them contains `n_freq` features. Each network is associated to a different `idx` number.

### Scale data
It is a common practice to provide scaled input to the neural network. For this reason a scaler vector is computed by calling the function `compute_normalizationVectorReal`. Three different arguments can be used for defining the `normalizeMethod` variable:
- localmax: each coefficient is scaled by its local maximum
- globalmax: all the coefficients are scaled by the same value which represent the global maximum
- None: no scaling is applied. The output vector contains ones.
Once that the scaling vector is known, the scaling is applied both to the training dataset and to the testing one.

### Train the network
The training of the neural network is carried out by calling the `model_train` function. The following inputs are requested:
- `idx`: integer, it is an identifier associated to the neural network. Thanks to this idx, more than one network can be trained in the same run and the weights can be stored in different files.
- `data_train`: dataset used for the training
- `data_valid`: dataset used for the validation
- `plotHistory` (otpional): boolean, plot  the trainig history when set to `True`

The resulting weights are saved on files.
**If this function is commented and if a file with weights of a previously trained network with the same parameters and dimensions is present, the network is loaded automatically from file.**

### Predict
After that the neural network has been trained, predictions of the coefficients can be extracted with the aid of the `spod_emulation.model_inference` routine. This receives as inputs:
- `idx`: integer, a value which identify a previously trained neural network (in this case 0, since we have only one neural network)
- `data_input`: data which are used to start the prediction. This array can have an arbitary length. The first `n_seq_in` data are copied in the output vector and used for predicting the next `n_seq_out` steps

The output consists in a vector which has the same dimensions of data_input and contains the predicted scaled coefficients.

### Rescale data

The predicted coefficients are then scaled back by calling `utils.denormalize_data`.

In [ ]:
for idx in range(n_modes):
	idx_x = list(range(idx, n_feature, n_modes))
    
	# scale data
	scaler  = \
		utils.compute_normalizationVector(coeffs_train['coeffs'][idx_x,:],normalizeMethod='localmax')
	data_train[:,:] = \
		utils.normalize_data(coeffs_train['coeffs'][idx_x,:], normalizationVec=scaler)
	data_test[:,:]  = \
		utils.normalize_data(coeffs_test['coeffs'][idx_x,:], normalizationVec=scaler)

    # train the network
	spod_emulation.model_train(idx,
		data_train=data_train, 
		data_valid=data_test,
		plotHistory=False
	)
    
	#predict 
	coeffs_tmp = spod_emulation.model_inference(
		idx,
		data_input=data_test
	)

	# rescale data
	coeffs_emul[idx_x,:] = utils.denormalize_data(coeffs_tmp, scaler)

### Reconstruct the solution

Now we have two distinct types of coefficients which we can use for reconstructing the snaptshots contained in X_test:
- `coeffs_test`: the ones which were obtained by projecting the snapshot on the SPOD basis
- `emul_coeffs`: the ones which were obtained with the prediction of the LSTM-based neural network.

Fields are reconstructed and stored in a proper numpy array by calling `reconstruct_data` and providing the following input:
- `coeffs`: the coefficients to be used for reconstructing the fields
- `phi_tilde`: a structure containing the modes computed in the `transform` function
- `time_mean`: the mean flow previously computed with the `transform` function

In [ ]:
# reconstruct solutions
phi_tilde = coeffs_train['phi_tilde']
time_mean = coeffs_train['time_mean']

proj_rec =spod.reconstruct_data(
		coeffs=coeffs_test['coeffs'][:,:], 
		phi_tilde=coeffs_train['phi_tilde'],
		time_mean=coeffs_train['time_mean']
	)

emulation_rec =spod.reconstruct_data(
		coeffs=coeffs_emul, 
		phi_tilde=coeffs_train['phi_tilde'],
		time_mean=coeffs_train['time_mean']
	)


## Output 

In the last section of the code, some routines are placed for visulizing some results and computing the errors.

`spod.printErrors`: compute and print L1, L2, and $L_{\inf}$ average norm error for both the learning and the projection error. 
In intput the following input are required:
- `field_test`: "true" solutions, it is a snapshot which belong to the original dataset 
- `field_proj`: fields reconstructed using the coeffs_test, from the comparison between this database and the one containing the true solutions we can evalute the projection error
- `field_emul`: fields reconstructed using the coeffs_emul; from the comparison between this database and the field_proj we can evalute the learning error; from the comparison between this database and the one containing the true solutions we can evalute the total error.
- `n_snaps`: number of snapshots on which the errors are evaluated 
- `n_offset`: offset

`spod.plot_eigs()`: plot the **eigenvalues in the complex plane**, that is part of the `postprocessing` module. 

`spod.plot_eigs_vs_period()`: plot the eigevalues as a function of the characteristic period. We need access to the array 'spod.freq'

`spod.plot_eigs_vs_frequency()`: plot the eigevalues as a function of the frequency. We need access to the array 'spod.freq'

`spod.plot_compareTimeSeries`: compare time series, it is here used for comparing actual coefficients and the learned ones. It requires in input: two time series, two labels of the time series, legendLocation(otpional), and the filename (optional)

`spod.generate_2D_subplot`: generate subpolts for visualizing the snapshots. It can show 1, 2 or 3 fields in the same frame. Inputs:
- `title1`: title associated to the first snapshot
- `title2`: title associated to the second snapshot (optional)
- `title3`: title associated to the third snapshot (optional)
- `var1`: 2D array of dimenions $20\times88$ that we want to plot
- `var2`: 2D array of dimenions $20\times88$ that we want to plot (optional)
- `var3`: 2D array of dimenions $20\times88$ that we want to plot (optional)
- `N_round`: number of decimals one wants to keep in the legend (optional)
- `path`: path where one wants to store the results(optional)
- `filename`: name of the file where to save the plot(optional)

In [ ]:
spod.printErrors(field_test=X_test, field_proj=proj_rec, field_emul=emulation_rec, n_snaps = 100, n_offset = 100)

#
spod.plot_eigs()
spod.plot_eigs_vs_period()
spod.plot_eigs_vs_frequency()

#routines for visualization
spod.plot_compareTimeSeries(
 	  serie1= coeffs_test['coeffs'][0,:],
 	  serie2= coeffs_emul[0,:],
 	  label1='test',
 	  label2='lstm',
 	  legendLocation = 'upper left',
 	  filename=None)

spod.generate_2D_subplot(
	title1='True solution', 
	title2='Projection-based solution', 
	title3='LSTM-based solution',
	var1=X_test[100,:,:], 
	var2=proj_rec[100,:,:,0], 
	var3=emulation_rec[100,:,:,0], 
	N_round=2, path='CWD', filename=None
)

